In [ ]:
// setup
#include <cstdlib>
#include <iostream>
#include <random>
#include <ctime>
using namespace std;

# Pseudo-random numbers in C++
Being able to work with numbers that are generated at random is a very powerful tool in programming. Such numbers can be used in simulating games (throwing dice, generating hands in card games), cryptography (read this easy to follow [article](http://www.loyalty.org/~schoen/rsa/) about RSA public key cryptography vulnerabilities caused by weak random number generators), solving difficult optimization problems with fast randomized algorithms ([notes](https://cse.iitkgp.ac.in/~abhij/course/theory/Algo2/Autumn20/slides/Randomized-Algo.pdf) about randomized algorithms, a little advanced but good to get an idea), etc.

The random numbers we are generating in programs are called **pseudo-random**: they are calculated using mathematical sequences, such as the following sequence called a *linear congruential sequence*:
$$ X_{n+1} = (a X_n + c) \mod m. $$
So, if we take $a=3$, $c=5$, and $m=7$, and if we choose the **seed** of the sequence $X_0=2$, we get
$$ X_1 = 11 \mod 7 = 4,\\ X_2 = 17 \mod 7 = 3,\\ X_3 = 14 \mod 7 = 0, \;\text{etc}.$$
We notice that the values $X_1, X_2, \ldots$, seem random (no pattern is evident) although they are not. It all boils down to choosing the right values for constants $a,c,m$. Of course, other mathematical sequences exist. 

## Generating random numbers in C++
### Method 1: the C method
1. Include ```<cstdlib>```
2. Initialize the value of the seed (optional). By default, the seed is 1.
3. Call ```rand()``` for the first random number, call it again for the second, etc.

In [ ]:
srand(3);  // initialize seed to 3
cout << rand() << " ";  // first random nr
cout << rand() << " ";  // second random nr ...

In [ ]:
srand(10);  // initialize seed to 10
cout << rand() << " ";  // first random nr
cout << rand() << " ";  // second random nr ...

In [ ]:
srand(3);  // initialize seed to 3
cout << rand() << " ";  // first random nr
cout << rand() << " ";  // second random nr ...

Notice that different seed values produce different sequences, but for the same seed, the sequence repeats. This is definitely not truly random. To improve the behaviour of their random number generators, programs use a different source of randomness, usually physical, for the seed. For example, mouse movement, network traffic, the internal clock. Using the clock, is quite common:

In [ ]:
#include <unistd.h>

for (int i=0; i< 10; i++) {
    cout << time(NULL) << ' ';  // time returns the number of seconds since a reference date/time (Jan 1, 1970)
    sleep(1);  // sleep() pauses the program for the nr of seconds provided as argument
                // without sleep, time() returns the same value. Try it!
}

In [ ]:
// every time this is executed, the generated sequence is different
srand(time(NULL));
for (int i=0; i<10; i++) {
    cout << rand() << ' ';
}

### Method 2: C++
Generators: C++ objects that generate sequences of numbers like rand(). Advantages over the C version: (1) there is a choice of mathematical functions to generate the seemingly random numbers, (2) one can use several independent random generators with different seeds instead of a single one (3) the methods are mathematically more robust.

In [ ]:
#include <random>

int seed = 3;
minstd_rand generator(seed);  // declare a variable (object) generator; 
                              // can use many generators, each with a different seed (check cplusplus.com)
                              // Just declare another generator object.
cout << generator() << ' ' << generator();

## Using numbers randomly generated
In applications, we may need random numbers that obey specific constraints. For example, they should be integers between 1 and 6 to simulate throwing dice, or they could only be 0 or 1 (tossing a coin). We need to write code to transform the generated numbers into the desired numbers.

### Method 1 - C
To generate a number between 1 and 6 we can apply $\mod 6$ and add 1:

In [ ]:
for (int i=0; i<10; i++) {
    cout << (rand() % 6 + 1) << ' ';
}

### Method 2 - C++
We can use a **distribution object**, for example the ```dice``` object defined below. This object takes a random number generator and distributes the values uniformly between the lower (ex 1) and upper (ex 6) bounds specified when the distribution object is declared and initialized.

In [ ]:
minstd_rand generator(4);  // seed 4
uniform_int_distribution<int> dice(1,6);

for (int i=0; i<10; i++) {
    cout << dice(generator) << ' ';  
}

### Appendix: seeding method 2 (C++)
When we include the ```<random>``` header, we have access to another class, the ```random_device``` class. This is a random number generator that uses a physical source of randomness, *if the hardware of the device running your code has this ability*. So, instead of using the ```time()``` function like demonstrated earlier, we can simply use this special random number generator. Notice that, unlike ```time()``` which returned the same value between successive *rapid* calls to ```time()```, ```random_device``` simply returns different values. So, we can use ```random_device``` to seed your pseudo-random sequences.

In [ ]:
random_device rd;
for (int i=0; i<10; i++) {
    cout << rd() << ' ';
}

In [ ]:
// creating a new random_device object does NOT start with the same sequence 
// (unlike the behaviour of other generators that use the same seed)
random_device rd2;
for (int i=0; i<10; i++) {
    cout << rd2() << ' ';
}

In [ ]:
// putting it all together:
// throwing a 6 face die 10 times
random_device rd;  // physical source of randomness for the seed
minstd_rand gen(rd());  // seed the generator
uniform_int_distribution<int> dice(1,6);  // initialize a distribution object

for (int i=0; i<10; i++) {
    cout << dice(gen) << ' ';
}

Other useful distribution objects: a rational between 0 and 1:

In [ ]:
random_device rd;
minstd_rand gen(rd());
uniform_real_distribution<double> dist(0.0,1.0);

for(int i=0; i<10; i++) {
    cout << dist(gen) << ' ';
}

## Exercises ##
1. Simulate 10 throws of a 20 face die (the faces are numbered 1-20). Write the code below using the C method.

2. Same as exercise 1 but use the C++ method.

3. Write code using the C method, then the C++ method, to generate a 5 card poker hand. There are a total of 52 cards in a poker game: there are 9 number cards numbered 2-10. Following are 4 figure cards: aces (represented by 11), jack (12), queen (13) and king(14). Each card belongs to one of four suites: spades (S), clubs (C), diamond (D), and hearts (H). Make sure your hand does not have two cards with the same number and suite. With this notation, the ace of clubs would be represented by 11C, the 4 of hearts by 4H, etc.

4. Enhance your code from exercise 3 to display 'a' for the ace (instead of 11), j for jack (instead of 12), q for queen (instead of 13), and k for king (instead of 14). This way, the ace of clubs would be represented by aC.